In [1]:
# https://pbpython.com/pdf-reports.html
# https://github.com/Kozea/WeasyPrint/tree/gh-pages/samples/report
# https://weasyprint.readthedocs.io/en/stable/tutorial.html#as-a-python-library

#--or create data model and make PPT to convert to PDF--
# https://community.esri.com/thread/179174

import pandas as pd
import numpy as np
from flask import *
import webbrowser
import os
import requests
import simplejson as json
from flask import jsonify
from collections import Counter


results = pd.read_csv('ENGAGE+2.0+State+of+Missouri+Informal+Supervisor+Upward+Feedback+Form_October+7,+2019_08.10.csv', skiprows=[1,2])
results = results.replace(np.nan, '', regex=True)
#print(results[['feedbackMDA','feedbackDMH']])
results['feedback'] = results[['feedbackMDA','feedbackDMH']].apply(lambda x: ''.join(x), axis=1)
names = results['feedback'].unique()
group = 'MDA'
Filtered_group = 'feedback'+group
Feedback_group = 'division'+group
names = results[Filtered_group].unique()
print(results.columns)
scoring = []
for name in names:
    if len(results[results['feedback']==name]) > 2:
        name1 = {}
        #print(name+":"+str(len(results[results['feedback']==name])))
        
        name1['name'] = name
        
        division = results[Feedback_group][results['feedback']==name].iloc[0]
        name1['division'] = division
        
        department = results['department'][results['feedback']==name].iloc[0]
        name1['department'] = department
        
        role = results['role'+group][results['feedback']==name].iloc[0]
        name1['role'] = role

        facility = results['facility'+group][results['feedback']==name].iloc[0]
        name1['facility'] = facility

        shift = results['shift'][results['feedback']==name].iloc[0]
        name1['shift'] = shift
        
        name1['countofresponse'] = results[results['feedback']==name].shape[0]

        
        score = results[['Core_Questions_1', 'Core_Questions_2',
       'Core_Questions_3', 'Core_Questions_4', 'Core_Questions_5',
       'Core_Questions_6']][results['feedback']==name]
        comments = results['comments'][results['feedback']==name]
        name1['comments']=(list(filter(None, comments.tolist())))
        overall = {'Sometimes': 0, 'Often': 0, 'Always': 0, 'Rarely': 0, 'Never': 0}
        for item in score.columns:
            try:
                overall = Counter(score[item].value_counts().to_dict()) + Counter(overall)
            except:
                overall = Counter(score[item].value_counts().to_dict())
            
            name1.update({str(item):score[item].value_counts(normalize=True).to_dict()})
        #print(overall)
        
        overall = dict(overall)
        name1['score'] = overall
        try:
            Always = overall['Always']
        except:
            Always = 0
        try:
            Often = overall['Often']
        except:
            Often = 0
        #print((Always+Often)/sum(overall.values()))
        name1['favScore'] = (Always+Often)/sum(overall.values())
        s = sum(overall.values())
        item_score ={}
        for k, v in overall.items():
            pct = v * 100.0 / s
            item_score[k] = pct
        #print(item_score)
        name1['item_score'] = item_score
        
        scoring.append(name1)

print(scoring)
print(len(scoring))
results = results.groupby('feedback')['ResponseId'].nunique()
print(results)








Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Authentication', 'department',
       'divisionMDA', 'divisionDMH', 'divisionDMH_3_TEXT', 'roleMDA',
       'roleDMH', 'facilityMDA', 'facilityDMH', 'shift', 'feedbackMDA',
       'feedbackDMH', 'Core_Questions_1', 'Core_Questions_2',
       'Core_Questions_3', 'Core_Questions_4', 'Core_Questions_5',
       'Core_Questions_6', 'comments', 'feedback'],
      dtype='object')
[{'name': 'WOLFE, MARK', 'division': 'MISSOURI STATE FAIR', 'department': 'Department of Agriculture', 'role': 'FISCAL & ADMINISTRATIVE MGR', 'facility': 'PETTIS COUNTY', 'shift': '', 'countofresponse': 6, 'comments': [], 'Core_Questions_1': {'Sometimes': 0.3333333333333333, 'Always': 0.333333333333

In [3]:
import sys  
import os  
import glob  
import win32com.client
import ast

from os import listdir
from os.path import isfile, join
mypath = r'C:\Users\willylee\Desktop\Projects\SRC Automation\State of Missouri Examples\report'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)

for filex in onlyfiles:
    location = os.path.join(mypath, filex)
    location = "r'"+str(location)+"'"
    def convert(files, formatType = 32):  
        powerpoint = win32com.client.Dispatch("Powerpoint.Application")  
        powerpoint.Visible = 1  
        for filename in files:  
            newname = os.path.splitext(filename)[0] + ".pdf"  
            deck = powerpoint.Presentations.Open(filename)  
            deck.SaveAs(newname, formatType)  
            deck.Close()  
        powerpoint.Quit()  

    files = glob.glob(ast.literal_eval(location)) # <--- ONLY CHANGE  
    convert(files)  

[]
